In [2]:
import os
import sys
import Levenshtein

import numpy as np

In [3]:
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

def ratio(s1, s2):
    if not s1 and not s2:
        return 1.0
    distance = levenshtein_distance(s1, s2)
    return (max(len(s1), len(s2)) - distance) / max(len(s1), len(s2))

# 示例用法
s1 = "kitten"
s2 = "sitting"
print(ratio(s1, ''))  # 输出相似度比率

0.0


In [1]:
def remove_duplicate_answers(querylist, answerlist):
    seen_answers = set()
    new_answers = []

    for query, answer in zip(querylist, answerlist):
        if answer not in seen_answers:
            seen_answers.add(answer)
            new_answers.append(answer)
        else:
            new_answers.append(None)  # 或者可以选择其他占位符，表示该答案被删除

    return new_answers

# 示例
querylist = ["问题1", "问题1", "问题2", "问题2"]
answerlist = ["答案1", "答案1", "答案2", "答案3"]
querylist_len = [2,2]
filtered_answers = remove_duplicate_answers(querylist, answerlist)
print(filtered_answers)  # 输出: ['答案1', '答案2', None, '答案3', None]

['答案1', None, '答案2', '答案3']


In [9]:
import copy
new_querylist = []
new_answerlist = []
new_querylist_len = copy.deepcopy(querylist_len)

# 记录每个查询对应的答案
query_to_answers = {}
for i in range(len(querylist)):
    if querylist[i] not in query_to_answers:
        query_to_answers[querylist[i]] = []
    query_to_answers[querylist[i]].append(answerlist[i])

# 处理每个查询及其对应的答案
for i,query in enumerate(query_to_answers):
    answers = query_to_answers[query]
    unique_answers = []
    seen_answers = set()
    query_count = 0  # 记录当前查询的重复次数

    for answer in answers:
        if answer not in seen_answers:
            # 检查当前答案与已添加的答案是否相似
            if all(not ratio(answer, unique_answer) >0.9 for unique_answer in unique_answers):
                unique_answers.append(answer)
                seen_answers.add(answer)
            else:
                new_querylist_len[i] -= 1
        else:
            new_querylist_len[i] -= 1



    # 更新新列表
    for answer in unique_answers:
        new_querylist.append(query)
        new_answerlist.append(answer)

In [14]:
probabilities=[0.1,0.9,0.3,0.7]
querylist = ["问题1", "问题1", "问题2", "问题2"]
answerlist = ["答案1", "答案1", "答案2", "答案3"]
querylist_len = [2,2]
# 初始化新的列表
new_querylist = []
new_answerlist = []
new_querylist_len = []
new_probabilities = []  # 新的概率列表

# 记录每个查询对应的答案及其概率
query_to_answers = {}
query_to_probs = {}
for i in range(len(querylist)):
    if querylist[i] not in query_to_answers:
        query_to_answers[querylist[i]] = []
        query_to_probs[querylist[i]] = []
    query_to_answers[querylist[i]].append(answerlist[i])
    query_to_probs[querylist[i]].append(probabilities[i])  # 假设 probabilities 是存储概率的列表

# 处理每个查询及其对应的答案
for query in query_to_answers:
    answers = query_to_answers[query]
    probs = query_to_probs[query]
    unique_answers = []
    unique_probs = []

    for j in range(len(answers)):
        answer = answers[j]
        prob = probs[j]
        # 检查当前答案与已添加的答案是否相似
        found_similar = False
        for k in range(len(unique_answers)):
            if ratio(answer, unique_answers[k])>0.9:
                unique_probs[k] += prob  # 合并概率
                found_similar = True
                break
        
        if not found_similar:
            # 如果没有找到相似的答案，添加到唯一答案列表
            unique_answers.append(answer)
            unique_probs.append(prob)

    # 更新新列表
    for i in range(len(unique_answers)):
        new_querylist.append(query)
        new_answerlist.append(unique_answers[i])
        new_querylist_len.append(answers.count(unique_answers[i]))  # 记录该答案的重复次数
        new_probabilities.append(unique_probs[i])  # 添加合并后的概率

# 更新原始列表
querylist = new_querylist
answerlist = new_answerlist
querylist_len = new_querylist_len
probabilities = new_probabilities  # 更新概率列表

In [16]:
unique_probs

[0.3, 0.7]

In [15]:
print(new_querylist,new_querylist_len,new_answerlist,probabilities)


['问题1', '问题2', '问题2'] [2, 1, 1] ['答案1', '答案2', '答案3'] [1.0, 0.3, 0.7]


In [2]:
from Levenshtein import ratio
def is_sim(str1:str, str2:str) -> bool:
    the_ratio = ratio(str1,str2)
    return the_ratio > 0.9

In [3]:
def merge(string_list:list):
    result=[]
    for string in string_list:
        if not any(is_sim(string, unique) for unique in result):
            result.append(string)
    return result

In [5]:
ratio("aaa",'')

0.0

In [4]:
string_list=["aaa", "bbb", "aaa","ccc"]
merge(string_list)

['aaa', 'bbb', 'ccc']

In [2]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('/data2/OpenLLMs/math-similarity/Bert-MLM_arXiv-MP-class_zbMath')
# model = SentenceTransformer('/data2/OpenLLMs/sentence-transformers/paraphrase-MiniLM-L6-v2')


/data1/home/jiawei/.conda/envs/open_reasonser/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def compute_similarity(sentences_pair):
    embeddings = model.encode(sentences_pair)
    embedding_1= model.encode(sentences_pair[0], convert_to_tensor=True)
    embedding_2 = model.encode(sentences_pair[1], convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding_1, embedding_2).cpu().numpy()[0][0]


In [26]:
embeddings.shape

(2, 384)

In [27]:
# import json
# import random

# # 读取 train.jsonl 文件
# with open('envs/MATH/dataset/train.jsonl', 'r', encoding='utf-8') as f:
#     lines = f.readlines()

# # 随机打乱行顺序
# random.shuffle(lines)

# # 选取前 500 行作为 for_gen.jsonl
# for_gen_lines = lines[:500]

# # 剩下的行作为 retain_train.jsonl
# retain_train_lines = lines[500:]

# # 将 for_gen_lines 写入 for_gen.jsonl 文件
# with open('envs/MATH/dataset/for_gen.jsonl', 'w', encoding='utf-8') as f:
#     f.writelines(for_gen_lines)

# # 将 retain_train_lines 写入 retain_train.jsonl 文件
# with open('envs/MATH/dataset/retain_train.jsonl', 'w', encoding='utf-8') as f:
#     f.writelines(retain_train_lines)

# print("文件分割完成！")

In [4]:
import re

def extract_candidate_actions(log_content):
    # 正则表达式匹配 Candidate action + 数字 的行
    pattern = re.compile(r'Candidate action \d: (.*)')
    
    # 用于存储提取的信息
    groups = []
    current_group = []
    
    for line in log_content.splitlines():
        match = pattern.match(line)
        if match:
            # 如果匹配到 Candidate action 行，提取冒号后的信息
            action_info = match.group(1)
            current_group.append(action_info)
        elif line.strip() == '********************************************************************************':
            # 如果遇到分隔符，将当前组添加到 groups 中，并重置 current_group
            if current_group:
                groups.append(current_group)
                current_group = []
    
    # 处理最后一个组
    if current_group:
        groups.append(current_group)
    
    return groups


In [5]:
def read_log_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [6]:
log_file_path="logs_terminal/vanila_mcts_math500.log"
log_content = read_log_file(log_file_path)

In [7]:
# 提取信息
groups = extract_candidate_actions(log_content)

# 打印结果
for i, group in enumerate(groups):
    if i > 5:
        break
    print(f"Group {i+1}:")
    for action in group:
        print(action)
    print()


Group 1:
{'action': 'We can use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right)$ to convert from rectangular coordinates to polar coordinates. ки\n', 'prob': 0.25739426139958294, 'num_token': 53, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right)$. ки\n', 'prob': 0.24839519181502717, 'num_token': 52, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right).$ ки\n', 'prob': 0.24785813497191633, 'num_token': 53, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we can use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right).$ ки\n', 'prob': 0.2463524118134735, 'num_token': 54, 'f

In [9]:
from Levenshtein import ratio

def calculate_levenshtein_ratios(group):
    """计算组内所有action之间的Levenshtein比率"""
    import ast
    ratios = []
    
    # 将字符串形式的字典转换为实际的字典对象
    actions = []
    for item in group:
        try:
            action_dict = ast.literal_eval(item)
            actions.append(action_dict['action'])
        except:
            continue
            
    # 计算每对action之间的比率
    for i in range(len(actions)):
        for j in range(i+1, len(actions)):
            # r = ratio(actions[i], actions[j])
            r = compute_similarity([actions[i], actions[j]])
            ratios.append(r)
            
    return ratios

# 为每个组计算比率
group_ratios_list = []
for i, group in enumerate(groups):
    if i > 5:
        break
    group_ratios = calculate_levenshtein_ratios(group)
    group_ratios_list.append(group_ratios)
    
# 计算每个组的统计信息并存储在变量中
import numpy as np
group_statistics = []

for idx, group_ratios in enumerate(group_ratios_list):
    if idx > 5:
        break
    group_stat = {}
    group_stat["组号"] = idx + 1
    if group_ratios:
        group_stat["平均相似度"] = np.mean(group_ratios)
        group_stat["最大相似度"] = np.max(group_ratios)
        group_stat["最小相似度"] = np.min(group_ratios)
        group_stat["相似度标准差"] = np.std(group_ratios)
    else:
        group_stat["无法计算相似度"] = "组内action不足"
    group_statistics.append(group_stat)

# 打印统计信息
for stat in group_statistics:
    print(stat)


{'组号': 1, '平均相似度': 0.9928252, '最大相似度': 0.9998011, '最小相似度': 0.9898933, '相似度标准差': 0.0034404902}
{'组号': 2, '平均相似度': 0.98817253, '最大相似度': 1.0000001, '最小相似度': 0.9822587, '相似度标准差': 0.008363396}
{'组号': 3, '平均相似度': 0.5680475, '最大相似度': 0.7986666, '最小相似度': 0.22002785, '相似度标准差': 0.21750563}
{'组号': 4, '平均相似度': 0.9083789, '最大相似度': 0.98200315, '最小相似度': 0.8614379, '相似度标准差': 0.052713}
{'组号': 5, '平均相似度': 0.79483134, '最大相似度': 0.9774318, '最小相似度': 0.6057078, '相似度标准差': 0.17773308}
{'组号': 6, '平均相似度': 0.9336293, '最大相似度': 0.9869937, '最小相似度': 0.8859408, '相似度标准差': 0.035924952}


In [13]:
for i in range(6):
    print(group_ratios_list[i])

[0.99027205, 0.9898933, 0.99032855, 0.9998011, 0.9938306, 0.9928255]
[1.0000001, 0.9822587, 0.9822587]
[0.31996503, 0.71736205, 0.7986666, 0.71037513, 0.22002785, 0.6418884]
[0.98200315, 0.8816957, 0.8614379]
[0.6057078, 0.9774318, 0.9728621, 0.6335184, 0.6127383, 0.9667295]
[0.9186898, 0.9709877, 0.9370198, 0.8859408, 0.9869937, 0.9021439]


In [11]:
from Levenshtein import ratio

def calculate_levenshtein_similarity_matrix(sentences):
    """计算句子列表中每对句子的Levenshtein相似度，并返回相似度矩阵"""
    num_sentences = len(sentences)
    similarity_matrix = np.zeros((num_sentences, num_sentences))
    
    for i in range(num_sentences):
        for j in range(num_sentences):
            similarity_matrix[i, j] = ratio(sentences[i], sentences[j])
    
    return similarity_matrix

In [14]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('/data2/OpenLLMs/sentence-transformers/paraphrase-MiniLM-L6-v2')
model = SentenceTransformer('/data2/OpenLLMs/math-similarity/Bert-MLM_arXiv-MP-class_zbMath')

sentences = [
    "To find $f(-2)$, we substitute $x=-2$ into the expression for $f(x)$:\n$f(-2) = \\frac{3(-2)-2}{(-2)-2} = \\frac{-6}{-4} = \\frac{3}{2}$. ки\n",
    "We can evaluate each term separately: ки\n",
    "Step 1: To find $f(-2)$, we substitute $x=-2$ into the expression for $f(x)$:\n$$f(-2) = \\frac{3(-2)-2}{-2-2} = \\frac{-6-2}{-4} = \\frac{-8}{-4} = 2.$$ ки\n",
    "We can evaluate $f(-2)$ by plugging in $x=-2$ into the equation:\n\\[f(-2) = \\frac{3(-2)-2}{-2-2} = \\frac{-6-2}{-4} = \\frac{-8}{-4} = 2.\\] ки\n"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

torch.Size([4, 4])


In [12]:

sentences = [
    "We want to find the least positive integer multiple of 30 that can be written with only the digits 0 and 2. ки\n",
    "The least positive integer multiple of 30 that can be written with only the digits 0 and 2 is $\\boxed{20}$. The answer is: 20 кики\n",
    "We can start by listing out the multiples of 30: 30, 60, 90, 120, 150, 180, ... ки\n",
    "To find the least positive integer multiple of 30 that can be written with only the digits 0 and 2, we need to find the smallest positive integer $n$ such that $10^{n-1} + 2$ is divisible by 30. ки\n",
    "To find the least positive integer multiple of 30, we need to find the smallest number that is divisible by both 2 and 3. ки\n",
    "Since we are looking for a multiple of 30, we need to find a number that is divisible by both 2 and 3. ки\n",
    "To find the least positive integer multiple of 30, we need to find the least positive integer that is divisible by both 3 and 10. ки\n"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.9786, 0.9421, 0.9869, 0.8330, 0.8740, 0.8348],
        [0.9786, 1.0000, 0.9238, 0.9744, 0.8174, 0.8741, 0.8190],
        [0.9421, 0.9238, 1.0000, 0.9474, 0.7952, 0.8448, 0.8022],
        [0.9869, 0.9744, 0.9474, 1.0000, 0.8732, 0.9046, 0.8736],
        [0.8330, 0.8174, 0.7952, 0.8732, 1.0000, 0.9625, 0.9947],
        [0.8740, 0.8741, 0.8448, 0.9046, 0.9625, 1.0000, 0.9637],
        [0.8348, 0.8190, 0.8022, 0.8736, 0.9947, 0.9637, 1.0000]])


In [27]:
probabilities=[0.1,0.9,0.3,0.7]
querylist = ["问题1", "问题1", "问题2", "问题2"]
answerlist = ["答案1", "答案1", "答案2", "答案3"]
querylist_len = [2,2]
# 初始化新的列表
new_querylist = []
new_answerlist = []
new_querylist_len = []
new_probabilities = []  # 新的概率列表

# 记录每个查询对应的答案及其概率
query_to_answers = {}
query_to_probs = {}
for i in range(len(querylist)):
    if querylist[i] not in query_to_answers:
        query_to_answers[querylist[i]] = []
        query_to_probs[querylist[i]] = []
    query_to_answers[querylist[i]].append(answerlist[i])
    query_to_probs[querylist[i]].append(probabilities[i])  # 假设 probabilities 是存储概率的列表

# 处理每个查询及其对应的答案
for query in query_to_answers:
    answers = query_to_answers[query]
    probs = query_to_probs[query]
    unique_answers = []
    unique_probs = []
    answer_counts = {}  # 用于记录每个答案的出现次数

    for j in range(len(answers)):
        answer = answers[j]
        prob = probs[j]
        # 检查当前答案与已添加的答案是否相似
        found_similar = False
        for k in range(len(unique_answers)):
            if ratio(answer, unique_answers[k])>0.9:
                unique_probs[k] += prob  # 合并概率
                answer_counts[unique_answers[k]] += 1  # 更新出现次数
                found_similar = True
                break
        
        if not found_similar:
            # 如果没有找到相似的答案，添加到唯一答案列表
            unique_answers.append(answer)
            unique_probs.append(prob)
            answer_counts[answer] = 1  # 初始化出现次数

    # 更新新列表
    for i in range(len(unique_answers)):
        new_querylist.append(query)
        new_answerlist.append(unique_answers[i])
        new_querylist_len.append(answer_counts[unique_answers[i]])  # 记录该答案的重复次数
        new_probabilities.append(unique_probs[i])  # 添加合并后的概率

# 更新原始列表
querylist = new_querylist
answerlist = new_answerlist
querylist_len = new_querylist_len
probabilities = new_probabilities  # 更新概率列表

In [4]:
probabilities=[0.1,0.9,0.3,0.7]
querylist = ["问题1", "问题1", "问题2", "问题2"]
answerlist = ["答案1", "答案1", "答案2", "答案3"]
# ... existing code ...
# 记录每个查询对应的答案及其概率
query_to_answers = {}
query_to_probs = {}
for i in range(len(querylist)):
    if querylist[i] not in query_to_answers:
        query_to_answers[querylist[i]] = []
        query_to_probs[querylist[i]] = []
    query_to_answers[querylist[i]].append(answerlist[i])
    query_to_probs[querylist[i]].append(probabilities[i])  # 假设 probabilities 是存储概率的列表

for query in query_to_answers:
    answers = query_to_answers[query]
    probs = query_to_probs[query]
    unique_answers = []
    unique_probs = []
    answer_counts = {}  # 用于记录每个答案的出现次数

    for j in range(len(answers)):
        answer = answers[j]
        prob = probs[j]
        # 检查当前答案与已添加的答案是否相似
        found_similar = False
        for k in range(len(unique_answers)):
            if ratio(answer, unique_answers[k]) > 0.9:
                unique_probs[k] += prob  # 合并概率
                answer_counts[unique_answers[k]] += 1  # 更新出现次数
                answerlist[j] = '<deleted>'  # 标记为已合并的答案
                probs[j] = float('-inf')  # 设置概率为负无穷
                found_similar = True
                break
        
        if not found_similar:
            # 如果没有找到相似的答案，添加到唯一答案列表
            unique_answers.append(answer)
            unique_probs.append(prob)
            answer_counts[answer] = 1  # 初始化出现次数

    # 更新新列表
    for i in range(len(unique_answers)):
        new_querylist.append(query)
        new_answerlist.append(unique_answers[i])
        new_querylist_len.append(answer_counts[unique_answers[i]])  # 记录该答案的重复次数
        new_probabilities.append(unique_probs[i])  # 添加合并后的概率
# ... existing code ...

NameError: name 'new_querylist' is not defined

In [28]:
print(new_querylist,new_querylist_len,new_answerlist,probabilities)


['问题1', '问题2', '问题2'] [2, 1, 1] ['答案1', '答案2', '答案3'] [1.0, 0.3, 0.7]


In [31]:
from collections import Counter

def calculate_querylist_len(querylist):
    # 统计每个查询出现的次数
    query_counts = Counter(querylist)
    
    # 获取所有独特的查询，并保持顺序
    unique_queries = []
    for query in querylist:
        if query not in unique_queries:
            unique_queries.append(query)
    
    # 生成querylist_len，表示每个独特查询在querylist中出现的次数
    querylist_len = [query_counts[query] for query in unique_queries]
    
    return querylist_len

# 示例
querylist = ['问题1', '问题2', '问题2', '问题2', '问题3']
querylist_len = calculate_querylist_len(querylist)
print(querylist_len)  # 输出: [1, 2]

[1, 3, 1]


In [15]:
similarity_matrix = calculate_levenshtein_similarity_matrix(sentences)
print(similarity_matrix)

[[1.         0.23595506 0.91034483 0.70758123]
 [0.23595506 1.         0.22680412 0.30939227]
 [0.91034483 0.22680412 1.         0.7440273 ]
 [0.70758123 0.30939227 0.7440273  1.        ]]
